In [2]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
import pickle

In [4]:
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [5]:
# Read data from files
train = pd.read_csv( os.path.join(DATADIR, 'labeledTrainData.tsv'), header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( os.path.join(DATADIR, "unlabeledTrainData.tsv"), header=0,  delimiter="\t", quoting=3 )

In [6]:
# Verify the number of reviews that were read (100,000 in total)
print "Read %d labeled train reviews " \
     "and %d unlabeled reviews\n" % (train["review"].size, unlabeled_train["review"].size )

Read 25000 labeled train reviews and 50000 unlabeled reviews



In [7]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
train.ix[0,]

id                                                    "5814_8"
sentiment                                                    1
review       "With all this stuff going down at the moment ...
Name: 0, dtype: object

### Get an even split train/test set from our labelled training set

because Kaggle's test set has no labels so we need our own labelled test set.

In [9]:
fracTrain = 0.5
nSamples = train.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = train.ix[order[:splitIndex],:]
test1 = train.ix[order[splitIndex:],:]

In [10]:
print train1.shape
print test1.shape

(12500, 3)
(12500, 3)


In [11]:
pickle.dump(train1, open('train1.p', 'wb'))
pickle.dump(test1, open('test1.p', 'wb'))
pickle.dump(unlabeled_train, open('unlabeled_train.p', 'wb'))

# Word2Vec inversion

Here is some stuff to split the labelled training set into positive and negative reviews. NOTE: takes a long time to run.

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into sentences and appends to a list of positive and negative sentences, respectively. Also does the same for the unlabelled training sentences.

In [12]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences
sentences_unlabelled = []  # Initialize an empty list of sentences

inxs_pos = np.where(train1["sentiment"] == 1)[0].tolist()
inxs_neg = np.where(train1["sentiment"] == 0)[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["review"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["review"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    
    
print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences_unlabelled += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)


Parsing sentences from unlabeled set


/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup

Now do the same for all sentences without splitting to build a vocabulary:

In [13]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train["review"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


Some stuff from Taddy's code (much simpler than Kaggle method for building Word2Vec models). 

In [14]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [15]:
basemodel.build_vocab(sentences) 

Train 2 models off of the base model (one pos & one neg).

In [16]:
from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

3250350

In [17]:
pickle.dump(models, open('models.p', 'wb'))

### Inversion of the distributed representations

At this point, we have 5 different word2vec language representations. Each 'model' has been trained conditional (i.e., limited to) text from a specific star rating. We will apply Bayes rule to go from p(text|stars) to p(stars|text).
For any new sentence we can obtain its likelihood (lhd; actually, the composite likelihood approximation; see the paper) using the score function in the word2vec class. We get the likelihood for each sentence in the first test review, then convert to a probability over star ratings. Every sentence in the review is evaluated separately and the final star rating of the review is an average vote of all the sentences. This is all in the following handy wrapper.

In [125]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [20]:
# read in the test set as a list of a list of words
docs = []
for review in test1["review"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

/home/sarahwie/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


In [126]:
probstest = docprob(docs[0:2], models)

In [127]:
probstest

,0,1
doc,,
0,0.446382,0.553618
1,0.324049,0.675951


### Test set example
As an example, we apply the inversion on the full test set.

In [82]:
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,models).astype(object)

In [85]:
probs

,0,1
doc,,
0,0.446382,0.553618
1,0.324049,0.675951
2,0.555172,0.444828
3,0.923994,0.076006
4,0.0584463,0.941554
5,0.323526,0.676474
6,0.507631,0.492369
7,0.343579,0.656421
8,0.759174,0.240826


In [84]:
pickle.dump(probs, open('probs.p', 'wb'))
pickle.dump(docs, open('docs.p','wb'))

A numpy array of ones for the number of docs that we're predicting:

In [86]:
predictions = np.ones((probs.shape[0]))

In [87]:
print np.shape(predictions)

(12500,)


In [88]:
predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [89]:
predictions

array([ 1.,  1.,  0., ...,  1.,  0.,  0.])

In [90]:
print predictions.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [91]:
print np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)

0.87376


In [92]:
pickle.dump(predictions, open('predictions.p', 'wb'))

### probsentences was created by commenting out the "prob = prob.groupby("doc").mean()" line in the function and then getting probs.

In [42]:
pickle.dump(probssentences, open('probssentences.p', 'wb'))

In [130]:
print predictions[0:10]

[ 1.  1.  0.  0.  1.  1.  0.  1.  0.  1.]
